In [1]:
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
# ｄｆとして保存済みのはずれ値処理データを読み込む
df= pd.read_pickle('ダミー処理①0218.pkl')

In [3]:
df.shape

(537533, 78)

In [4]:
df.head(3)

,場所,馬名,距離,走破タイム,レース番号,騎手名,斤量,馬番,馬体重,上がり3Fタイム,...,美→中京,美→中山,美→京都,美→函館,美→小倉,美→新潟,美→札幌,美→東京,美→福島,美→阪神
0,札幌,サンダビューク,1000,59.3,7,石川倭,58.0,6,454,35.9,...,False,False,False,False,False,False,True,False,False,False
1,札幌,フクノワカバ,1000,59.3,7,鮫島克駿,54.0,5,446,36.1,...,False,False,False,False,False,False,False,False,False,False
2,札幌,クロンヌドラレーヌ,1000,59.3,7,佐々木大,54.0,12,452,35.1,...,False,False,False,False,False,False,True,False,False,False


In [5]:
#NAないか調べる
df.isnull().sum(axis=0)

場所       0
馬名       0
距離       0
走破タイム    0
レース番号    0
        ..
美→新潟     0
美→札幌     0
美→東京     0
美→福島     0
美→阪神     0
Length: 78, dtype: int64

In [6]:
#どのようなコラムがあるか表示
print(df.columns.to_list())

['場所', '馬名', '距離', '走破タイム', 'レース番号', '騎手名', '斤量', '馬番', '馬体重', '上がり3Fタイム', '調教師', '所属地', '現馬主名', '生産者名', '父馬名', '母馬名', '母の父馬名', 'レースID', '仮予測', '生後日数', 'レースごとの平均賞金', '稍', '良', '重', '牝', '牡', 11, 15, 23, 39, 43, 67, 115, 131, 147, 163, 179, 195, '栗毛', '白毛', '芦毛', '青毛', '青鹿', '鹿毛', '黒鹿', '地→中山', '地→京都', '地→函館', '地→小倉', '地→新潟', '地→札幌', '地→東京', '地→福島', '地→阪神', '外→中京', '外→中山', '外→東京', '外→阪神', '栗→中京', '栗→中山', '栗→京都', '栗→函館', '栗→小倉', '栗→新潟', '栗→札幌', '栗→東京', '栗→福島', '栗→阪神', '美→中京', '美→中山', '美→京都', '美→函館', '美→小倉', '美→新潟', '美→札幌', '美→東京', '美→福島', '美→阪神']


# 親名を数値化（生涯総獲得賞金）

In [7]:
# ｄｆとして保存済みの「馬」データを読み込む
uma = pd.read_pickle('馬.pkl')

In [8]:
uma.head(1)

,馬名,血統登録番号,性別,所属,調教師名,種牡馬名,母名,母の父名,馬主名,生産者名,...,芝成績3着数,芝成績着外数,ダート成績1着数,ダート成績2着数,ダート成績3着数,ダート成績着外数,データ作成日,抹消,登録日,抹消日
0,アーヴァイン,15106091,牡,栗,浅見秀一,キンシャサノキセキ,マザイ,Fasliyev,林正道,服部牧場,...,0,8,0,0,0,1,190416,抹消,170629,180828


In [9]:
#コラムを確認
print(uma.columns.to_list())

['馬名', '血統登録番号', '性別', '所属', '調教師名', '種牡馬名', '母名', '母の父名', '馬主名', '生産者名', '毛色', '生年月日', '本賞金', '収得賞金', '前走馬体重', '全成績1着数', '全成績2着数', '全成績3着数', '全成績着外数', '芝成績1着数', '芝成績2着数', '芝成績3着数', '芝成績着外数', 'ダート成績1着数', 'ダート成績2着数', 'ダート成績3着数', 'ダート成績着外数', 'データ作成日', '抹消', '登録日', '抹消日']


In [10]:
#データフレームumaを必要な列だけのdfにする
columns=['馬名','本賞金']
uma=uma[columns]
uma.shape

(99999, 2)

In [11]:
uma.head(2)

,馬名,本賞金
0,アーヴァイン,310
1,アーヴィング,0


In [12]:
#重複を調べる
uma['馬名'].is_unique

False

In [13]:
# 特定の列においてユニークでない行を削除
列名 = '馬名'
uma = uma[~uma.duplicated(subset=[列名])]

In [14]:
uma.shape

(98370, 2)

In [15]:
# '馬名'列を'父馬名'に変更
uma.rename(columns={'馬名': '父馬名'}, inplace=True)

# 結果を表示
print(uma.head(2))

      父馬名  本賞金
0  アーヴァイン  310
1  アーヴィング    0


In [16]:
# '本賞金'列を'父の賞金'に変更
uma.rename(columns={'本賞金': '父の賞金'}, inplace=True)

# 結果を表示
print(uma.head(2))

      父馬名  父の賞金
0  アーヴァイン   310
1  アーヴィング     0


In [17]:
import pandas as pd

# 父馬名をプライマリキーとして「父の賞金」列を df に追加
df2 = pd.merge(df, uma[['父馬名', '父の賞金']], on='父馬名', how='left')

# 結果のデータフレーム df2を表示
df2.shape

(537533, 79)

In [18]:
df2.head(7)

,場所,馬名,距離,走破タイム,レース番号,騎手名,斤量,馬番,馬体重,上がり3Fタイム,...,美→中山,美→京都,美→函館,美→小倉,美→新潟,美→札幌,美→東京,美→福島,美→阪神,父の賞金
0,札幌,サンダビューク,1000,59.3,7,石川倭,58.0,6,454,35.9,...,False,False,False,False,False,True,False,False,False,NaN
1,札幌,フクノワカバ,1000,59.3,7,鮫島克駿,54.0,5,446,36.1,...,False,False,False,False,False,False,False,False,False,NaN
2,札幌,クロンヌドラレーヌ,1000,59.3,7,佐々木大,54.0,12,452,35.1,...,False,False,False,False,False,True,False,False,False,65140.0
3,札幌,アカザ,1000,59.4,7,鷲頭虎太,53.0,10,444,36.1,...,False,False,False,False,False,True,False,False,False,51630.0
4,札幌,ナムラジョシュア,1000,59.5,7,長岡禎仁,56.0,4,470,35.7,...,False,False,False,False,False,False,False,False,False,NaN
5,札幌,カップッチョ,1000,59.6,7,黛弘人,58.0,2,488,35.9,...,False,False,False,False,False,True,False,False,False,29535.0
6,札幌,グランデ,1000,59.7,7,今村聖奈,55.0,11,470,36.3,...,False,False,False,False,False,False,False,False,False,NaN


In [19]:
#最頻値
df2['父の賞金'].value_counts()

父の賞金
38400.0    9291
94540.0    5426
53720.0    4749
76720.0    4388
42060.0    3763
           ... 
2930.0        1
23220.0       1
21210.0       1
24360.0       1
4505.0        1
Name: count, Length: 269, dtype: int64

In [20]:
#NaNの数
df2['父の賞金'].isna().sum()

423044

In [21]:
#同様に、'母馬名',も数値化する

# '父馬名'列を'母馬名'に変更
uma.rename(columns={'父馬名': '母馬名'}, inplace=True)

# '父の賞金'列を'母の賞金列'に変更
uma.rename(columns={'父の賞金': '母の賞金'}, inplace=True)

# 結果を表示
print(uma.head(2))

      母馬名  母の賞金
0  アーヴァイン   310
1  アーヴィング     0


In [22]:
# 母馬名をプライマリキーとして「父の賞金」列を df に追加
df2 = pd.merge(df2, uma[['母馬名', '母の賞金']], on='母馬名', how='left')

# 結果のデータフレーム df2を表示
df2.head(7)

,場所,馬名,距離,走破タイム,レース番号,騎手名,斤量,馬番,馬体重,上がり3Fタイム,...,美→京都,美→函館,美→小倉,美→新潟,美→札幌,美→東京,美→福島,美→阪神,父の賞金,母の賞金
0,札幌,サンダビューク,1000,59.3,7,石川倭,58.0,6,454,35.9,...,False,False,False,False,True,False,False,False,NaN,NaN
1,札幌,フクノワカバ,1000,59.3,7,鮫島克駿,54.0,5,446,36.1,...,False,False,False,False,False,False,False,False,NaN,3665.0
2,札幌,クロンヌドラレーヌ,1000,59.3,7,佐々木大,54.0,12,452,35.1,...,False,False,False,False,True,False,False,False,65140.0,14376.0
3,札幌,アカザ,1000,59.4,7,鷲頭虎太,53.0,10,444,36.1,...,False,False,False,False,True,False,False,False,51630.0,NaN
4,札幌,ナムラジョシュア,1000,59.5,7,長岡禎仁,56.0,4,470,35.7,...,False,False,False,False,False,False,False,False,NaN,336.0
5,札幌,カップッチョ,1000,59.6,7,黛弘人,58.0,2,488,35.9,...,False,False,False,False,True,False,False,False,29535.0,NaN
6,札幌,グランデ,1000,59.7,7,今村聖奈,55.0,11,470,36.3,...,False,False,False,False,False,False,False,False,NaN,380.0


In [23]:
#最頻値
df2['母の賞金'].value_counts()

母の賞金
0.0       41167
50.0       2990
700.0      2905
180.0      1700
75.0       1679
          ...  
1295.0        1
4765.0        1
4960.0        1
3824.0        1
6251.0        1
Name: count, Length: 2822, dtype: int64

In [24]:
#NaNの数
df2['母の賞金'].isna().sum()

375849

In [25]:
#同様に、'母の父馬名'も数値化する

# '母馬名'列を'母の父馬名'に変更
uma.rename(columns={'母馬名': '母の父馬名'}, inplace=True)

# '母の賞金'列を'母父の賞金列'に変更
uma.rename(columns={'母の賞金': '母父の賞金'}, inplace=True)

# 結果を表示
print(uma.head(2))

    母の父馬名  母父の賞金
0  アーヴァイン    310
1  アーヴィング      0


In [26]:
# 母の父馬名をプライマリキーとして「母父の賞金」列を df に追加
df2 = pd.merge(df2, uma[['母の父馬名', '母父の賞金']], on='母の父馬名', how='left')

# 結果のデータフレーム df2を表示
df2.head(7)

,場所,馬名,距離,走破タイム,レース番号,騎手名,斤量,馬番,馬体重,上がり3Fタイム,...,美→函館,美→小倉,美→新潟,美→札幌,美→東京,美→福島,美→阪神,父の賞金,母の賞金,母父の賞金
0,札幌,サンダビューク,1000,59.3,7,石川倭,58.0,6,454,35.9,...,False,False,False,True,False,False,False,NaN,NaN,NaN
1,札幌,フクノワカバ,1000,59.3,7,鮫島克駿,54.0,5,446,36.1,...,False,False,False,False,False,False,False,NaN,3665.0,94540.0
2,札幌,クロンヌドラレーヌ,1000,59.3,7,佐々木大,54.0,12,452,35.1,...,False,False,False,True,False,False,False,65140.0,14376.0,NaN
3,札幌,アカザ,1000,59.4,7,鷲頭虎太,53.0,10,444,36.1,...,False,False,False,True,False,False,False,51630.0,NaN,NaN
4,札幌,ナムラジョシュア,1000,59.5,7,長岡禎仁,56.0,4,470,35.7,...,False,False,False,False,False,False,False,NaN,336.0,NaN
5,札幌,カップッチョ,1000,59.6,7,黛弘人,58.0,2,488,35.9,...,False,False,False,True,False,False,False,29535.0,NaN,NaN
6,札幌,グランデ,1000,59.7,7,今村聖奈,55.0,11,470,36.3,...,False,False,False,False,False,False,False,NaN,380.0,NaN


In [27]:
#最頻値
df2['母父の賞金'].value_counts()

母父の賞金
38400.0     5300
132400.0    2767
94540.0     1467
53720.0     1277
72800.0     1191
            ... 
47100.0        2
590.0          2
52200.0        1
3600.0         1
30320.0        1
Name: count, Length: 67, dtype: int64

In [28]:
#NaNの数
df2['母父の賞金'].isna().sum()

521524

In [29]:
# '父馬名''母馬名' '母の父馬名'を削除
df2 = df2.drop(['父馬名','母馬名', '母の父馬名'], axis=1)
df2.shape

(537533, 78)

# 調教師名を数値化（生涯1着数）

In [30]:
# ｄｆとして保存済みの「調教師」データを読み込む
tyokyoshi = pd.read_pickle('調教師.pkl')

In [31]:
tyokyoshi.info

<bound method DataFrame.info of       順位 名前(ターゲット内表記) フルネーム(JRA-VANデータ内表記)  読み(半角仮名/欧字)               欧字表記名  \
0      1         和田勇介                和田 勇介     ﾜﾀﾞ ﾕｳｽｹ         Yusuke Wada   
1      2         和田雄二                和田 雄二     ﾜﾀﾞ ﾕｳｼﾞ           Yuji Wada   
2      3         和田正一               和田 正一郎  ﾜﾀﾞ ｼｮｳｲﾁﾛｳ      Shoichiro Wada   
3      4         矢野英一                矢野 英一      ﾔﾉ ｴｲｲﾁ         Eiichi Yano   
4      5         矢嶋大樹                矢嶋 大樹    ﾔｼﾞﾏ ﾀﾞｲｷ        Daiki Yajima   
..   ...          ...                  ...          ...                 ...   
191  192         鮫島克也                鮫島 克也     ｻﾒｼﾏ ｶﾂﾔ   Katsuya Sameshima   
192  193         九日俊光                九日 俊光     ｸﾆﾁ ﾄｼﾐﾂ  Toshimitsu Kunichi   
193  194         北村欣也                北村 欣也     ｷﾀﾑﾗ ｷﾝﾔ      Kinya Kitamura   
194  195         川田孝好                川田 孝好    ｶﾜﾀﾞ ﾀｶﾖｼ    Takayoshi Kawada   
195  196         角川秀樹                角川 秀樹   ｶｸｶﾞﾜ ﾋﾃﾞｷ     Hideki Kakugawa   

     所属  年齢        

In [32]:
#コラムを確認
print(tyokyoshi.columns.to_list())

['順位', '名前(ターゲット内表記)', 'フルネーム(JRA-VANデータ内表記)', '読み(半角仮名/欧字)', '欧字表記名', '所属', '年齢', '誕生日', '免許取得年', 'キャリア', '本年着度数', '1着数', '2着数', '3着数', '着外数', '総数', '勝率', '連対率', '複勝率', '前年着度数', '前年1着', '前年2着', '前年3着', '前年着外', '前年総数', '前年勝率', '前年連対率', '前年複勝率', '生涯着度数', '生涯1着', '生涯2着', '生涯3着', '生涯着外', '生涯総数', '生涯勝率', '生涯連対率', '生涯複勝率', 'コード', '現役・抹消', 'チェック', '勝数前年比', '総レース数前年比', 'コメント', '本年順位(平地)', '前年順位(平地)']


In [33]:
#重複を調べる
tyokyoshi['名前(ターゲット内表記)'].is_unique

True

In [34]:
#データフレームtyokyoshiを必要な列だけのdfにする
tyokyoshi=tyokyoshi[['名前(ターゲット内表記)','生涯1着']]
tyokyoshi.head(7)

,名前(ターゲット内表記),生涯1着
0,和田勇介,79
1,和田雄二,109
2,和田正一,207
3,矢野英一,259
4,矢嶋大樹,0
5,森一誠,0
6,村田一誠,10


In [35]:
# '名前(ターゲット内表記)'列を'調教師'に変更
tyokyoshi.rename(columns={'名前(ターゲット内表記)': '調教師'}, inplace=True)
# '生涯1着'列名を'調教師点'に変更
tyokyoshi.rename(columns={'生涯1着': '調教師点'}, inplace=True)
# 結果を表示
print(tyokyoshi.head(2))

    調教師  調教師点
0  和田勇介    79
1  和田雄二   109


C:\Users\newpc\AppData\Local\Temp\ipykernel_14728\654854529.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tyokyoshi.rename(columns={'名前(ターゲット内表記)': '調教師'}, inplace=True)
C:\Users\newpc\AppData\Local\Temp\ipykernel_14728\654854529.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tyokyoshi.rename(columns={'生涯1着': '調教師点'}, inplace=True)


In [36]:
# '調教師'をプライマリキーとして'調教師点'列を df に追加
df2 = pd.merge(df2, tyokyoshi[['調教師','調教師点']], on='調教師', how='left')

# 結果のデータフレーム df2を表示
df2.head(7)

,場所,馬名,距離,走破タイム,レース番号,騎手名,斤量,馬番,馬体重,上がり3Fタイム,...,美→小倉,美→新潟,美→札幌,美→東京,美→福島,美→阪神,父の賞金,母の賞金,母父の賞金,調教師点
0,札幌,サンダビューク,1000,59.3,7,石川倭,58.0,6,454,35.9,...,False,False,True,False,False,False,NaN,NaN,NaN,88.0
1,札幌,フクノワカバ,1000,59.3,7,鮫島克駿,54.0,5,446,36.1,...,False,False,False,False,False,False,NaN,3665.0,94540.0,43.0
2,札幌,クロンヌドラレーヌ,1000,59.3,7,佐々木大,54.0,12,452,35.1,...,False,False,True,False,False,False,65140.0,14376.0,NaN,295.0
3,札幌,アカザ,1000,59.4,7,鷲頭虎太,53.0,10,444,36.1,...,False,False,True,False,False,False,51630.0,NaN,NaN,23.0
4,札幌,ナムラジョシュア,1000,59.5,7,長岡禎仁,56.0,4,470,35.7,...,False,False,False,False,False,False,NaN,336.0,NaN,235.0
5,札幌,カップッチョ,1000,59.6,7,黛弘人,58.0,2,488,35.9,...,False,False,True,False,False,False,29535.0,NaN,NaN,243.0
6,札幌,グランデ,1000,59.7,7,今村聖奈,55.0,11,470,36.3,...,False,False,False,False,False,False,NaN,380.0,NaN,286.0


In [37]:
#最頻値
df2['調教師点'].value_counts()

調教師点
336.0    6808
259.0    6038
631.0    5972
821.0    5318
512.0    4921
         ... 
25.0      109
20.0       67
9.0        24
2.0        12
0.0         2
Name: count, Length: 133, dtype: int64

In [38]:
#NaNの数
df2['調教師点'].isna().sum()

283407

In [39]:
# '調教師'を削除
df2 = df2.drop(['調教師'], axis=1)
df2.shape

(537533, 78)

In [40]:
#コラムを確認
print(df2.columns.to_list())

['場所', '馬名', '距離', '走破タイム', 'レース番号', '騎手名', '斤量', '馬番', '馬体重', '上がり3Fタイム', '所属地', '現馬主名', '生産者名', 'レースID', '仮予測', '生後日数', 'レースごとの平均賞金', '稍', '良', '重', '牝', '牡', 11, 15, 23, 39, 43, 67, 115, 131, 147, 163, 179, 195, '栗毛', '白毛', '芦毛', '青毛', '青鹿', '鹿毛', '黒鹿', '地→中山', '地→京都', '地→函館', '地→小倉', '地→新潟', '地→札幌', '地→東京', '地→福島', '地→阪神', '外→中京', '外→中山', '外→東京', '外→阪神', '栗→中京', '栗→中山', '栗→京都', '栗→函館', '栗→小倉', '栗→新潟', '栗→札幌', '栗→東京', '栗→福島', '栗→阪神', '美→中京', '美→中山', '美→京都', '美→函館', '美→小倉', '美→新潟', '美→札幌', '美→東京', '美→福島', '美→阪神', '父の賞金', '母の賞金', '母父の賞金', '調教師点']


In [41]:
# dfのNaNを全て0に変換
df2.fillna(0, inplace=True)

In [42]:
# データフレームをpickleファイルとして保存
df2.to_pickle('名前を数値化0218.pkl')

# 騎手を数値化（生涯1着数）

In [43]:
# ｄｆとして保存済みの「調教師」データを読み込む
jockey = pd.read_pickle('騎手.pkl')

In [44]:
#コラムを確認
print(jockey.columns.to_list())

['順位', '名前(ターゲット内表記)', 'フルネーム(JRA-VANデータ内表記)', '読み(半角仮名/欧字)', '欧字表記名', '所属', '年齢', '誕生日', '免許取得年', 'キャリア', '本年着度数', '1着数', '2着数', '3着数', '着外数', '総数', '勝率', '連対率', '複勝率', '前年着度数', '前年1着', '前年2着', '前年3着', '前年着外', '前年総数', '前年勝率', '前年連対率', '前年複勝率', '生涯着度数', '生涯1着', '生涯2着', '生涯3着', '生涯着外', '生涯総数', '生涯勝率', '生涯連対率', '生涯複勝率', 'コード', '現役・抹消', 'チェック', '勝数前年比', '総レース数前年比', 'コメント', '本年順位(平地)', '前年順位(平地)']


In [45]:
#重複を調べる
jockey['名前(ターゲット内表記)'].is_unique

True

In [46]:
#データフレームtyokyoshiを必要な列だけのdfにする
jockey=jockey[['名前(ターゲット内表記)','生涯1着']]
jockey.head(7)

,名前(ターゲット内表記),生涯1着
0,吉田豊,1289
1,吉田隼人,1157
2,横山琉人,56
3,横山典弘,2922
4,横山武史,505
5,横山和生,349
6,山田敬士,40


In [47]:
# '名前(ターゲット内表記)'列を'騎手名'に変更
jockey.rename(columns={'名前(ターゲット内表記)': '騎手名'}, inplace=True)
# '生涯1着'列名を'騎手点'に変更
jockey.rename(columns={'生涯1着': '騎手点'}, inplace=True)
# 結果を表示
print(jockey.head(2))

    騎手名   騎手点
0  吉田豊　  1289
1  吉田隼人  1157


In [48]:
# '騎手名'をプライマリキーとして'騎手名点'列を df に追加
df2 = pd.merge(df2, jockey[['騎手名','騎手点']], on='騎手名', how='left')

# 結果のデータフレーム df2を表示
df2.head(7)

,場所,馬名,距離,走破タイム,レース番号,騎手名,斤量,馬番,馬体重,上がり3Fタイム,...,美→新潟,美→札幌,美→東京,美→福島,美→阪神,父の賞金,母の賞金,母父の賞金,調教師点,騎手点
0,札幌,サンダビューク,1000,59.3,7,石川倭,58.0,6,454,35.9,...,False,True,False,False,False,0.0,0.0,0.0,88.0,NaN
1,札幌,フクノワカバ,1000,59.3,7,鮫島克駿,54.0,5,446,36.1,...,False,False,False,False,False,0.0,3665.0,94540.0,43.0,412.0
2,札幌,クロンヌドラレーヌ,1000,59.3,7,佐々木大,54.0,12,452,35.1,...,False,True,False,False,False,65140.0,14376.0,0.0,295.0,49.0
3,札幌,アカザ,1000,59.4,7,鷲頭虎太,53.0,10,444,36.1,...,False,True,False,False,False,51630.0,0.0,0.0,23.0,14.0
4,札幌,ナムラジョシュア,1000,59.5,7,長岡禎仁,56.0,4,470,35.7,...,False,False,False,False,False,0.0,336.0,0.0,235.0,84.0
5,札幌,カップッチョ,1000,59.6,7,黛弘人,58.0,2,488,35.9,...,False,True,False,False,False,29535.0,0.0,0.0,243.0,NaN
6,札幌,グランデ,1000,59.7,7,今村聖奈,55.0,11,470,36.3,...,False,False,False,False,False,0.0,380.0,0.0,286.0,71.0


In [49]:
#最頻値
df2['騎手点'].value_counts()

騎手点
1460.0    9227
1759.0    7504
1431.0    7492
965.0     7118
1809.0    6898
          ... 
1.0        104
0.0        102
4.0         71
63.0        69
281.0       44
Name: count, Length: 116, dtype: int64

In [50]:
#NaNの数
df2['騎手点'].isna().sum()

227845

In [51]:
df2.shape

(537533, 79)

In [52]:
# '騎手名'を削除
df2 = df2.drop(['騎手名'], axis=1)
df2.shape

(537533, 78)

# 生産者を数値化（生涯1着数）

In [53]:
# ｄｆとして保存済みの'生産者'データを読み込む
seisansya = pd.read_pickle('生産者.pkl')

In [54]:
seisansya.shape

(1808, 26)

In [55]:
#コラムを確認
print(seisansya.columns.to_list())

['生産者コード', '生産者名', '本年1着', '本年2着', '本年3着', '本年4着', '本年5着', '本年6着以下', '本年総数', '本年勝率', '本年連対率', '本年複勝率', '本年本賞金計(万円)', '本年1走当本賞金(万円)', '通算1着', '通算2着', '通算3着', '通算4着', '通算5着', '通算6着以下', '通算総数', '通算勝率', '通算連対率', '通算複勝率', '通算本賞金計(万円)', '通算1走当本賞金(万円)']


In [56]:
#重複を調べる
seisansya['生産者名'].is_unique

False

In [57]:
# 特定の列においてユニークでない行を削除
列名 = '生産者名'
seisansya = seisansya[~seisansya.duplicated(subset=[列名])]

In [58]:
seisansya.shape

(1728, 26)

In [59]:
#データフレームseisansyaを必要な列だけのdfにする
seisansya=seisansya[['生産者名','通算本賞金計(万円)']]
seisansya.head(7)

,生産者名,通算本賞金計(万円)
0,ノーザンファーム,7171309.0
1,社台ファーム,5906921.0
2,社台コーポレーション白老ファーム,3607832.0
3,千代田牧場,1159554.0
4,下河辺牧場,1677316.0
5,ビッグレッドファーム,1485146.0
6,岡田スタツド,1217896.0


In [60]:
#コラムを確認
print(df2.columns.to_list())

['場所', '馬名', '距離', '走破タイム', 'レース番号', '斤量', '馬番', '馬体重', '上がり3Fタイム', '所属地', '現馬主名', '生産者名', 'レースID', '仮予測', '生後日数', 'レースごとの平均賞金', '稍', '良', '重', '牝', '牡', 11, 15, 23, 39, 43, 67, 115, 131, 147, 163, 179, 195, '栗毛', '白毛', '芦毛', '青毛', '青鹿', '鹿毛', '黒鹿', '地→中山', '地→京都', '地→函館', '地→小倉', '地→新潟', '地→札幌', '地→東京', '地→福島', '地→阪神', '外→中京', '外→中山', '外→東京', '外→阪神', '栗→中京', '栗→中山', '栗→京都', '栗→函館', '栗→小倉', '栗→新潟', '栗→札幌', '栗→東京', '栗→福島', '栗→阪神', '美→中京', '美→中山', '美→京都', '美→函館', '美→小倉', '美→新潟', '美→札幌', '美→東京', '美→福島', '美→阪神', '父の賞金', '母の賞金', '母父の賞金', '調教師点', '騎手点']


In [61]:
# '通算本賞金計(万円)'列名を'生産者点'に変更
seisansya.rename(columns={'通算本賞金計(万円)': '生産者点'}, inplace=True)
# 結果を表示
print(seisansya.head(2))

       生産者名       生産者点
0  ノーザンファーム  7171309.0
1    社台ファーム  5906921.0


In [62]:
# '生産者名'をプライマリキーとして'生産者点'列を df に追加
df2 = pd.merge(df2, seisansya[['生産者名','生産者点']], on='生産者名', how='left')

# 結果のデータフレーム df2を表示
df2.head(7)

,場所,馬名,距離,走破タイム,レース番号,斤量,馬番,馬体重,上がり3Fタイム,所属地,...,美→札幌,美→東京,美→福島,美→阪神,父の賞金,母の賞金,母父の賞金,調教師点,騎手点,生産者点
0,札幌,サンダビューク,1000,59.3,7,58.0,6,454,35.9,美,...,True,False,False,False,0.0,0.0,0.0,88.0,NaN,72498.0
1,札幌,フクノワカバ,1000,59.3,7,54.0,5,446,36.1,栗,...,False,False,False,False,0.0,3665.0,94540.0,43.0,412.0,404263.0
2,札幌,クロンヌドラレーヌ,1000,59.3,7,54.0,12,452,35.1,美,...,True,False,False,False,65140.0,14376.0,0.0,295.0,49.0,5906921.0
3,札幌,アカザ,1000,59.4,7,53.0,10,444,36.1,美,...,True,False,False,False,51630.0,0.0,0.0,23.0,14.0,194921.0
4,札幌,ナムラジョシュア,1000,59.5,7,56.0,4,470,35.7,栗,...,False,False,False,False,0.0,336.0,0.0,235.0,84.0,236503.0
5,札幌,カップッチョ,1000,59.6,7,58.0,2,488,35.9,美,...,True,False,False,False,29535.0,0.0,0.0,243.0,NaN,238440.0
6,札幌,グランデ,1000,59.7,7,55.0,11,470,36.3,栗,...,False,False,False,False,0.0,380.0,0.0,286.0,71.0,54188.0


In [63]:
#最頻値
df2['生産者点'].value_counts()

生産者点
5906921.0    29358
7171309.0    28254
903635.0      7501
3607832.0     7437
1159554.0     7400
             ...  
8960.0           1
4330.0           1
3368.0           1
278.0            1
857.0            1
Name: count, Length: 1259, dtype: int64

In [64]:
#NaNの数
df2['生産者点'].isna().sum()

76619

In [65]:
# '生産者名'を削除
df2 = df2.drop(['生産者名'], axis=1)
df2.shape

(537533, 78)

In [66]:
# dfのNaNを全て0に変換
df2.fillna(0, inplace=True)

In [67]:
# データフレームをpickleファイルとして保存
df2.to_pickle('名前を数値化0218.pkl')